# Create an empty Ubuntu VM on Azure

Install HAGrid

```
pip install hagrid
```

__NOTE__: Just in case there are some issues with a VM we should provision are few extra, here we use node_count 12 for a session of 10 users

Run hagrid launch with these arguments:

```
hagrid launch to azure --image_name=domain_0.7.0 --jupyter --ansible_extras="install=false,aa_demo=true" --node_count 12
```

- Use a new unique resource group for this session like: aa-test-1
- Choose the location where your demo participants will be located, e.g. `eastus`, `westus` etc
- Choose an 8 core machine like `Standard_D8s_v3`
- Set the username to `azureuser`
- Choose password and then `n` to auto-generate password
- Set an easy to remember 12 character password like: `Mflpdemo2022`
- What ever you enter into Repo and Branch will be ignored

![ip_address](img/hagrid_bare_vm.png)

After it is finished you should see this message

![ip_address](img/hagrid_bare_vm_output.png)

Now run this to get JSON containing the information for all the VMs:

```
cat ~/.hagrid/host_ips.json
```

In [ ]:
import os
import json

# paste the path to host ip json here
HOST_IP_PATH = "~/.hagrid/host_ips.json"
HOST_IP_PATH = os.path.expanduser(HOST_IP_PATH)

with open(HOST_IP_PATH) as fp:
    host_ips = json.loads(fp.read())

In [ ]:
host_ips

In [ ]:
# update TOTAL_PARTICIPANTS
# use the total participants not the total machines, e.g. 10 not 12
# as this is used to calculate the data split assignment
TOTAL_PARTICIPANTS = 10

In [ ]:
# add names and emails so we can send out the links with details via email

In [ ]:
email_institution_map = {
    "madhava@openmined.org": "OpenMined U"
}
ip_institution_map = {}

In [ ]:
participants = {
    "Madhava": "madhava@openmined.org",
    # "Kyoko": "kyoko@openmined.org",
    # "Andrew": "andrew@openmined.org",
    # "Shubham": "shubham@openmined.org",
}

In [ ]:
# optionally add names or emails here which will be printed below to help keep track of assignment

assert len(participants) <= TOTAL_PARTICIPANTS, "TOTAL_PARTICIPANTS should be less than or equal to the length of participant list"
print("Total participants:", len(participants))

If you need to re-partition the MedNIST dataset and create new data subsets, switch to [prepare MedNIST dataset notebook](02-prepare-datasets-MedNIST.ipynb).

In [ ]:
import requests
datasets =  ["MedNIST","TissueMNIST"]
curr_dataset_name = datasets[1]
DATASET_INFO_FILEPATH = f"https://raw.githubusercontent.com/OpenMined/datasets/main/{curr_dataset_name}/dataset.json"

def get_dataset_urls():
    
    data_subset_urls = []
    
    response = requests.get(DATASET_INFO_FILEPATH)
    data_subset_info = response.json()
    DATASET_REPO_URL = f"https://raw.githubusercontent.com/OpenMined/datasets/main/{curr_dataset_name}/subsets/"
    
    for dataset_name in data_subset_info.values():
        url = DATASET_REPO_URL + dataset_name
        data_subset_urls.append(url)
        
    return data_subset_urls

In [ ]:
def check_ip_port(host_ip: str, port: int) -> bool:
    import socket
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex((host_ip, port))
        sock.close()
        if result == 0:
            return True
    except Exception:
        pass
    return False

In [ ]:
def get_icon(status: bool) -> str:
    return "✅" if status else "❌"

In [ ]:
def check_hosts_ready(host_ips: dict) -> None:
    for host in host_ips["host_ips"]:
        print("-----------------------")
        host_ip = host["ip_address"]
        # make sure the containers are not running
        http_up = check_ip_port(host_ip=host_ip, port=80)
        print(f"{get_icon(not http_up)} Containers Off {host_ip}:80")
        
        # make sure jupyter notebooks is up
        jupyter_up = check_ip_port(host_ip=host_ip, port=8888)
        print(f"{get_icon(jupyter_up)} Jupyter Up {host_ip}:8888")
        
        # make sure SSH is up
        ssh_up = check_ip_port(host_ip=host_ip, port=22)
        print(f"{get_icon(ssh_up)} SSH Up {host_ip}:22")

        print()
        all_status = (not http_up) and jupyter_up and ssh_up
        print(f"{get_icon(all_status)} Node {host_ip} Ready!")
        print("-----------------------")
        print()

In [ ]:
import paramiko

def save_dataset_url_to_vm(host_ip: str, dataset_url: str, institution: str = ""):
    print("running save_dataset_url_to_vm with:", host_ip, dataset_url, institution)
    ssh_client = paramiko.SSHClient()

    # To avoid an "unknown hosts" error. Solve this differently if you must...
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    # This mechanism uses a password.
    # Get it from cli args or a file or hard code it, whatever works best for you
    password = "Adastrademo2022"
    username = "azureuser"

    ssh_client.connect(hostname=host_ip, username=username, password=password)

    filename = "notebooks/medical-federated-learning-program/network-operators/update_dataset_url.py"
    command = f"cd /home/om/PySyft && sudo python {filename} {dataset_url}"
    
    if institution:
        command = f"cd /home/om/PySyft && sudo python {filename} {dataset_url} '{institution}'"

    # In order to inspect the exit code
    # you need go under paramiko's hood a bit
    # rather than just using "ssh_client.exec_command()"
    # chan = ssh_client.get_transport().open_session()
    print("Executing command...........")
    stdin,stdout,stderr = ssh_client.exec_command(command)

    print(stdout.readlines())

    print("Executing command.........DONE")



    # exit_status = chan.recv_exit_status()

    if stderr.readlines():
        print(f"Failed for IpAddress: {host_ip}")
    else:
        print(f"Successfully Update dataset url for {host_ip}")

In [ ]:
check_hosts_ready(host_ips)

In [ ]:
def output_user_details(host_ips: dict, participants: dict[str, str] = {}) -> None:
    
    host_to_dataset_url_map = dict()

    notebook_path = "medical-federated-learning-program/data-owners/data-owners-presentation.ipynb"
    print("===============================")
    print("Medical Federated Learning Program")
    print("===============================")
    print()
    print("Send to each participant")
    print()
    if TOTAL_PARTICIPANTS > len(host_ips["host_ips"]):
        raise Exception(
            f"TOTAL_PARTICIPANTS: {TOTAL_PARTICIPANTS} is less than VM count: {len(host_ips['host_ips'])}"
        )
    partition = 0
    dataset_urls = get_dataset_urls()
    num_of_urls = len(dataset_urls)
    for host in host_ips["host_ips"]:
        partition += 1
        if partition <= len(participants):
            print(f"Hi {list(participants.keys())[partition - 1]},")
        if partition <= TOTAL_PARTICIPANTS:
            print("These are your Session Details:")
        else:
            print("Spare Session Details:")
        print("-------------------------------")
        # print(f"VM Username: {host['username']}")
        # print(f"VM Password: {host['password']}")
        # print(f"VM IP Address: {host['ip_address']}")
        
        dataset_url = dataset_urls[partition%num_of_urls]
        host_to_dataset_url_map[host['ip_address']] = dataset_url
        if partition <= len(participants):
            participant_email = list(participants.values())[partition - 1]
            if participant_email in email_institution_map:
                institution = email_institution_map[participant_email]
                ip_institution_map[host['ip_address']] = institution
        print("DATASET:")
        print(f"{dataset_url}")
        notebook_url = (f"http://{host['ip_address']}:8888/lab/tree/notebooks/{notebook_path}"
            f"?token={host['jupyter_token']}")
        print()
        print(f"👉🏽 Start Here:")
        print(
            notebook_url
        )
        print()
    
    return host_to_dataset_url_map

In [ ]:
host_to_dataset_url_map = output_user_details(host_ips, participants)

In [ ]:
for host_ip, dataset_url in host_to_dataset_url_map.items():
    institution = ""
    if host_ip in ip_institution_map:
        institution = ip_institution_map[host_ip]
    save_dataset_url_to_vm(host_ip=host_ip, dataset_url=dataset_url, institution=institution)

In [ ]:
addresses_sent_to = []

In [ ]:
addresses_sent_to

In [ ]:
!pip install sendgrid

In [ ]:
from_address='research@openmined.org'
# from_address='madhava@openmined.org' # test

In [ ]:
SENDGRID_API_KEY = ""

In [ ]:
template = """
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html
  data-editor-version="2"
  class="sg-campaigns"
  xmlns="http://www.w3.org/1999/xhtml"
>
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta
      name="viewport"
      content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1"
    />
    <style type="text/css">
      html {
        background: #ffffff;
        font-family: "Chivo", sans-serif;
      }
    </style>
  </head>
  <body>
    <table width="100%">
      <tr>
        <td style="padding-left: 20px; padding-right: 20px" align="center">
          <img
            src="http://cdn.mcauto-images-production.sendgrid.net/919ca6f839ef4a13/3297075a-7366-4dc8-b0a5-c3525725d64f/1183x266.jpg"
            width="100%" alt="OpenMined: The Medical Federated Learning Program" 
            style="text-align: center"
          />
        </td>
      </tr>
    </table>
    <table width="100%">
      <tr>
        <td style="padding-left: 20px; padding-right: 20px">
"""

close_template = """
        </td>
      </tr>
    </table>
  </body>
</html>

"""

In [ ]:
email_contents = """
<p>
    Hi {firstName}, <br />Welcome to The Medical Federated Learning Program.<br />To
    help with today&#39;s session we have prepared some details for you,
    outlined below:
</p>
<h3>Session Details</h3>
<p>
    <strong>Jupyter Notebook</strong><br />&#128071;&#127997; Begin by
    clicking this link to start your interactive Jupyter Notebook<br /><a
    target="_blank" href="{notebook_url}"
    >Open Interactive Notebook</a
    >
</p>
<p>
    <strong>My Sample Dataset</strong><br />&#128206; This is the sample
    dataset you will be using for today&#39;s exercise. You will need to Copy
    and Paste this dataset&#39;s URL(listed below) later during the demo.<br /><br />
    {dataset_url}
</p>
<p>
    We hope you have a wonderful session.<br />If you need any assistance
    after the demo you can always reach out with technical issues by going to
    our Slack
    <a target="_blank" href="https://openmined.slack.com/archives/C6EEFN3A8">#support</a>
    channel, or for any other concerns you can email our team at
    <a target="_blank" href="mailto:support@openmined.org">support@openmined.org</a>.
</p>
<p>--<br />Regards,<br />The OpenMined Research Team
<br />openmined.org</p>
"""

In [ ]:
from io import StringIO
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
text_contents = strip_tags(email_contents)

In [ ]:
text_contents

In [ ]:
def make_email_html(template, content_template, close_template, person, notebook_url, dataset_url) -> str:
    content = content_template.replace("{firstName}", person)
    content = content.replace("{notebook_url}", notebook_url)
    content = content.replace("{dataset_url}", dataset_url)
    return f"{template}{content}{close_template}".replace("\n", "")

In [ ]:
make_email_html(template, email_contents, close_template, "Madhava", "http://google.com", "http://openmined.org")

In [ ]:
def send_email(person, email, host, notebook_url, dataset_url) -> bool:
    print("send email to", person, email, host)
    html_content = make_email_html(template, email_contents, close_template, person, notebook_url, dataset_url)
    # using SendGrid's Python Library
    # https://github.com/sendgrid/sendgrid-python
    import os
    from sendgrid import SendGridAPIClient
    from sendgrid.helpers.mail import Mail

    text_with_unsubscribe = text_contents + "\nUnsubscribe: unsubscribe@openmined.org"
    html_with_unsubscribe = html_content + '<br /><br /><a href="{{{unsubscribe}}}">Click here to unsubscribe.</a>'

    message = Mail(
        from_email=from_address,
        to_emails=email,
        subject="OpenMined: The Medical Federated Learning Program",
        plain_text_content=text_with_unsubscribe,
        html_content=html_with_unsubscribe)
    try:
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        response = sg.send(message)
        if response.status_code == 202:
            return True
        else:
            print(response.status_code)
            print(response.body)
            print(response.headers)
            return False
    except Exception as e:
        print(e.message)
        return False
    return True

In [ ]:
def send_emails(host_ips: dict, participants: dict[str, str] = {}) -> None:
    notebook_path = "medical-federated-learning-program/data-owners/data-owners-presentation.ipynb"
    print("===============================")
    print("Medical Federated Learning Program")
    print("===============================")
    print()
    print("Send to each participant")
    print()
    if TOTAL_PARTICIPANTS > len(host_ips["host_ips"]):
        raise Exception(
            f"TOTAL_PARTICIPANTS: {TOTAL_PARTICIPANTS} is less than VM count: {len(host_ips['host_ips'])}"
        )
    partition = 0

    dataset_urls = get_dataset_urls()
    num_of_urls = len(dataset_urls)
    for host in host_ips["host_ips"]:
        try:
            partition += 1
            notebook_url = f"http://{host['ip_address']}:8888/lab/tree/notebooks/{notebook_path}?token={host['jupyter_token']}"
            dataset_url = dataset_urls[partition%num_of_urls]
            if partition <= len(participants):
                person_name = list(participants.keys())[partition - 1]
                print(f"Hi {person_name},")
                person_email = participants[person_name]
                if person_email not in addresses_sent_to:
                    if send_email(person_name, person_email, host, notebook_url, dataset_url):
                        addresses_sent_to.append(person_email)
                else:
                    print(f"Already emailed: {person_name} {person_email}")
            if partition <= TOTAL_PARTICIPANTS:
                print("These are your Session Details:")
            else:
                print("Spare Session Details:")
            print("-------------------------------")
            # print(f"VM Username: {host['username']}")
            # print(f"VM Password: {host['password']}")
            # print(f"VM IP Address: {host['ip_address']}")
            print(f"{dataset_url}")

            print()
            print(f"👉🏽 Start Here:")
            print(notebook_url)
            print()
        except Exception as e:
            print(f"Failed to send details for partition: {partition}", e)

In [ ]:
send_emails(host_ips, participants)